In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
#working actual directory
#df = pd.read_csv(r'D:\Shyam\Documents\MUSER\test.csv',encoding='latin-1')
#df.info()

df = pd.read_csv(r'D:\Shyam\Documents\MUSER\Extraction\music-analysis-user2.csv',encoding='latin')

#df = pd.read_csv(r'D:\Shyam\Documents\MUSER\Extraction\music-analysis-user2.csv',encoding='utf-8-sig')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 57 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   user_id                                     714 non-null    object 
 1   record_id                                   714 non-null    object 
 2   player_event_type                           559 non-null    object 
 3   ui_event_type                               154 non-null    object 
 4   event_current_time_in_milliseconds          713 non-null    float64
 5   event_nano_time                             713 non-null    float64
 6   event_seek_position_in_milliseconds         713 non-null    float64
 7   event_start_time                            0 non-null      float64
 8   event_elapsed_time                          0 non-null      float64
 9   song_album_id                               652 non-null    float64
 10  song_album_nam

In [3]:
#sorting the current timestamp in ascending order
df.sort_values('event_current_time_in_milliseconds', ascending=True, inplace=True)

In [4]:
#adding a new column event_current_time_ms_to_date to the csv 
df['event_current_time_ms_to_date'] = pd.to_datetime(df['event_current_time_in_milliseconds'], unit='ms')

In [5]:
#remove the empty valued audio_volume_data column
df = df.dropna(axis=0, subset=['audio_volume_data'])
df.shape

(713, 58)

In [6]:
#function to extract minimum volume from an array of audio_volume_data
def pull_vol_min(i):
    a,b,c,d= i.split(',')
    return c

In [7]:
df['min_volume'] =df['audio_volume_data'].apply(pull_vol_min)
df['min_volume'] = df['min_volume'].str.split('=').str[1]

In [8]:
#function to extract current volume from an array of audio_volume_data
def pull_vol_curr(i):
    a,b,c,d,e= i.split()
    start = b.find('[')
    end = b.find(',')
    substring = b[start+1:end]
    return substring

In [9]:
df['curent_volume'] = df.audio_volume_data.apply(pull_vol_curr)
df['curent_volume'] = df['curent_volume'].str.split('=').str[1]

In [11]:
df.head()

,user_id,record_id,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_nano_time,event_seek_position_in_milliseconds,event_start_time,event_elapsed_time,song_album_id,...,device_info_ignoring_battery_optimizations,device_info_power_save_mode_enabled,device_info_sdk_version,device_info_sdk_version_int,device_info_talk_back_enabled,device_info_timestamp,event_current_time_in_iso_utc,event_current_time_ms_to_date,min_volume,curent_volume
0,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,11-259a0ea4-a35c-4995-adf5-d61280b816fb,PAUSE,NaN,1.631730e+12,6.700520e+11,12867.0,NaN,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-15T18:18:34.262Z,2021-09-15 18:20:00,0,8
539,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,74-da03ca46-0018-4500-84c3-548593182eaf,SEEK,NaN,1.631730e+12,1.125900e+12,228136.0,NaN,NaN,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-15T18:26:10.110Z,2021-09-15 18:20:00,0,8
540,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,75-d5878e20-3589-4b58-93b3-71968876a2de,SEEK,NaN,1.631730e+12,1.125970e+12,243816.0,NaN,NaN,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-15T18:26:10.183Z,2021-09-15 18:20:00,0,8
541,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,76-e0f3c76b-53d7-43c9-b8f7-71e6ff208be4,SEEK,NaN,1.631730e+12,1.125990e+12,248735.0,NaN,NaN,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-15T18:26:10.201Z,2021-09-15 18:20:00,0,8
542,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,77-5fa1fb8b-3411-4983-a108-83ff8cca688f,SEEK,NaN,1.631730e+12,1.126060e+12,266876.0,NaN,NaN,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-15T18:26:10.270Z,2021-09-15 18:20:00,0,8


In [10]:
#filtering out the records that are greater than the minimum volume value
df= df[ df['curent_volume'] > df['min_volume'] ]

In [11]:
# shorten the final file by only keeping required columns in the final csv file
user_behavior_data = df[["user_id", "record_id", "song_id", "song_name", "player_event_type", "ui_event_type",  "event_current_time_in_milliseconds", "event_seek_position_in_milliseconds", "song_album_name", "song_artist_name", "song_duration", "song_name", "audio_volume_data","event_current_time_ms_to_date", "min_volume", "curent_volume"]]

In [44]:
user_behavior_data.head()

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,song_duration,song_name,audio_volume_data,event_current_time_ms_to_date,min_volume,curent_volume
31,YNjF5zjgyFXJYHNadzvVfaYJuTE2,16-d35a09c5-88b5-4eb2-b9de-73a8c4992088,17.0,"Moonlight Sonata No. 14, 2nd Movement",PAUSE,NaN,1.623340e+12,2272.0,Tropical Sweets,Bernard Roberts,420258.0,"Moonlight Sonata No. 14, 2nd Movement","VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15
64,YNjF5zjgyFXJYHNadzvVfaYJuTE2,10-f6219398-92c1-4ae8-bff8-24c512285b5d,20.0,Flight of the Bumblebee,NaN,PLAY,1.623340e+12,0.0,Tropical Sweets,Andre Previn and the London Symphony Orchestra,102818.0,Flight of the Bumblebee,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15
116,YNjF5zjgyFXJYHNadzvVfaYJuTE2,33-4caddc90-6610-4578-a9bc-19397e9ea028,NaN,NaN,NaN,VOLUME_NO_CHANGE,1.623340e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15
115,YNjF5zjgyFXJYHNadzvVfaYJuTE2,32-cf3a66e7-60a2-4b67-bd81-a47b5f3e399d,NaN,NaN,NaN,VOLUME_DOWN,1.623340e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15
114,YNjF5zjgyFXJYHNadzvVfaYJuTE2,31-b85574e7-2efa-4d52-bc52-996eedc017db,NaN,NaN,NaN,VOLUME_NO_CHANGE,1.623340e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15


In [32]:
## Original Code
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  listening_time = 0
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if event_type != "PLAY" and user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY": #this is covering same song pause-play
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)
    if (event_type == "PAUSE" or event_type == "NEXT" or event_type == "PREV") and (user_behavior_data.loc[idx-1, "player_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY"):#when idx-1 is play and idx is either play or next or prev
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)

C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [12]:
# Changes to the original code
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  listening_time = 0
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if event_type != "PLAY" and user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY": #this is covering same song pause-play
        listening_time = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)
    if (event_type == "PAUSE" or event_type == "NEXT" or event_type == "PREV") and (user_behavior_data.loc[idx-1, "player_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY"):#when idx-1 is play and idx is either play or next or prev
        listening_time = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)

C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
user_behavior_data.head()

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,song_duration,song_name,audio_volume_data,event_current_time_ms_to_date,min_volume,curent_volume,listening_time
31,YNjF5zjgyFXJYHNadzvVfaYJuTE2,16-d35a09c5-88b5-4eb2-b9de-73a8c4992088,17.0,"Moonlight Sonata No. 14, 2nd Movement",PAUSE,NaN,1.623340e+12,2272.0,Tropical Sweets,Bernard Roberts,420258.0,"Moonlight Sonata No. 14, 2nd Movement","VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15,NaN
64,YNjF5zjgyFXJYHNadzvVfaYJuTE2,10-f6219398-92c1-4ae8-bff8-24c512285b5d,20.0,Flight of the Bumblebee,NaN,PLAY,1.623340e+12,0.0,Tropical Sweets,Andre Previn and the London Symphony Orchestra,102818.0,Flight of the Bumblebee,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15,NaN
116,YNjF5zjgyFXJYHNadzvVfaYJuTE2,33-4caddc90-6610-4578-a9bc-19397e9ea028,NaN,NaN,NaN,VOLUME_NO_CHANGE,1.623340e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15,NaN
115,YNjF5zjgyFXJYHNadzvVfaYJuTE2,32-cf3a66e7-60a2-4b67-bd81-a47b5f3e399d,NaN,NaN,NaN,VOLUME_DOWN,1.623340e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15,NaN
114,YNjF5zjgyFXJYHNadzvVfaYJuTE2,31-b85574e7-2efa-4d52-bc52-996eedc017db,NaN,NaN,NaN,VOLUME_NO_CHANGE,1.623340e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=15, volumeMax=1...",2021-06-10 15:46:40,0,15,NaN


In [17]:
# TRIAL ONE
# Duration of song that is at least half lister
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  ui_event = user_behavior_data.loc[idx, "ui_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  song_duration_time = user_behavior_data.loc[idx, "song_duration"]
  event_seek = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] 
  half_song_duration = 0
  if ui_event != "PLAY" and user_behavior_data.loc[idx - 1, "ui_event_type"] == "PLAY":
    if event_seek >= song_duration_time/2:
           half_song_duration = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
           user_behavior_data.loc[idx, "half_song_duration"] = half_song_duration/(1000*60)

C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
user_behavior_data

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,song_duration,song_name,audio_volume_data,event_current_time_ms_to_date,min_volume,curent_volume,half_song_duration
0,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,11-259a0ea4-a35c-4995-adf5-d61280b816fb,594.0,Daybreak from the Florida Suite,PAUSE,NaN,1.631730e+12,12867.0,Tropical Sweets,Sir Thomas Beecham,344869.0,Daybreak from the Florida Suite,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
539,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,74-da03ca46-0018-4500-84c3-548593182eaf,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,228136.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
540,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,75-d5878e20-3589-4b58-93b3-71968876a2de,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,243816.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
541,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,76-e0f3c76b-53d7-43c9-b8f7-71e6ff208be4,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,248735.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
542,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,77-5fa1fb8b-3411-4983-a108-83ff8cca688f,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,266876.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,696-e03e12a2-e246-4218-83d0-2dc8625a070a,402.0,Wake Up Little Susie,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2","Everly Brothers, The",122331.0,Wake Up Little Susie,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN
515,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,695-809de392-082e-4446-be42-72a919481038,399.0,Oh! My Papa,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2",Eddie Fisher,189179.0,Oh! My Papa,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN
514,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,694-83d8564c-59a4-48ca-bbca-3c15e4eb45b2,403.0,Allez-Vous-En,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2",Kay Starr,155350.0,Allez-Vous-En,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN
526,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,704-7cf63f9a-0f5d-4788-9286-ddef05218a65,400.0,In the Chapel in the Moonlight,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2",Kitty Kallen,173819.0,In the Chapel in the Moonlight,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN


In [12]:
# TRIAL TWO
# Duration for half listener
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  ui_event = user_behavior_data.loc[idx, "ui_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  song_duration_time = user_behavior_data.loc[idx, "song_duration"]
  event_seek = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  half_song_duration = 0  
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if user_behavior_data.loc[idx, "ui_event_type"] == "PLAY":
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            half_song_duration = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "half_song_duration"] = half_song_duration/(1000*60)

C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
user_behavior_data

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,song_duration,song_name,audio_volume_data,event_current_time_ms_to_date,min_volume,curent_volume,half_song_duration
0,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,11-259a0ea4-a35c-4995-adf5-d61280b816fb,594.0,Daybreak from the Florida Suite,PAUSE,NaN,1.631730e+12,12867.0,Tropical Sweets,Sir Thomas Beecham,344869.0,Daybreak from the Florida Suite,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
539,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,74-da03ca46-0018-4500-84c3-548593182eaf,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,228136.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
540,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,75-d5878e20-3589-4b58-93b3-71968876a2de,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,243816.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
541,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,76-e0f3c76b-53d7-43c9-b8f7-71e6ff208be4,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,248735.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
542,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,77-5fa1fb8b-3411-4983-a108-83ff8cca688f,531.0,All the Man That I Need [Live from Welcome Hom...,SEEK,NaN,1.631730e+12,266876.0,Live: Her Greatest Performances,Whitney Houston,307461.0,All the Man That I Need [Live from Welcome Hom...,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-09-15 18:20:00,0,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,696-e03e12a2-e246-4218-83d0-2dc8625a070a,402.0,Wake Up Little Susie,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2","Everly Brothers, The",122331.0,Wake Up Little Susie,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN
515,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,695-809de392-082e-4446-be42-72a919481038,399.0,Oh! My Papa,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2",Eddie Fisher,189179.0,Oh! My Papa,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN
514,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,694-83d8564c-59a4-48ca-bbca-3c15e4eb45b2,403.0,Allez-Vous-En,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2",Kay Starr,155350.0,Allez-Vous-En,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN
526,B38ZkYwQMaPyJ8Mb9IER5Vk0o522,704-7cf63f9a-0f5d-4788-9286-ddef05218a65,400.0,In the Chapel in the Moonlight,PLAY,NaN,1.632910e+12,0.0,"Decade of Hits: The 50's, Disc 2",Kitty Kallen,173819.0,In the Chapel in the Moonlight,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2021-09-29 10:06:40,0,9,NaN


In [14]:
#path = r'A:\MUSER\data\cleaned_data\mergedFiles\dummy\analysis_Code'
path = r'D:\Shyam\Documents\MUSER\Analysis'

user_behavior_data.to_csv(os.path.join(path,'muser_user_cleaned_dataset.csv'), index=False, encoding='utf-8-sig')